In [1]:
import pandas as pd
import requests
pd.options.display.max_columns = 999

### Scrape data from all positions

In [2]:
url = 'https://www.pro-football-reference.com/years/2021/rushing.htm'
resp = requests.get(url)
resp

output = open('./nfl_data/2021_rushing_stats.xls', 'wb')
output.write(resp.content)
output.close()

In [3]:
position = ['rushing', 'passing', 'receiving']

for element in position:
    url = f'https://www.pro-football-reference.com/years/2021/{element}.htm'
    resp = requests.get(url)
    
    output = open(f'./nfl_data/2021_{element}_stats.xls', 'wb')
    output.write(resp.content)
    output.close()

In [4]:
df_wr = pd.read_html('./nfl_data/2021_receiving_stats.xls')
df_wr = pd.DataFrame(df_wr[0]) # Saves df var to dataframe
#df_wr = df_rb.droplevel(0, axis=1) # Removes first level column

df_qb = pd.read_html('./nfl_data/2021_passing_stats.xls')
df_qb = pd.DataFrame(df_qb[0]) # Saves df var to dataframe
#df_qb = df_rb.droplevel(0, axis=1) # Removes first level column

df_rb = pd.read_html('./nfl_data/2021_rushing_stats.xls')
df_rb = pd.DataFrame(df_rb[0]) # Saves df var to dataframe
df_rb = df_rb.droplevel(0, axis=1) # Removes first level column

### Clean Data and set column types to int/float

In [5]:
def drop_rows(position):
    for header in position.columns:
        index_list = position.loc[(position[header] == header)].index
        position.drop(labels=index_list, axis=0, inplace = True)
        return
    
drop_rows(df_rb)
drop_rows(df_wr)
drop_rows(df_qb)

In [6]:
int_cols = ['Att', 'Yds', 'TD', '1D', 'Lng', 'Fmb', 'G', 'GS']
float_cols = ['Y/A', 'Y/G']

for int_col in int_cols:
    df_rb[f'{int_col}'] = df_rb[f'{int_col}'].astype(int)

for float_col in float_cols:
    df_rb[f'{float_col}'] = df_rb[f'{float_col}'].astype(float)

df_rb['Player'] = df_rb['Player'].map(lambda x: x.lstrip('*').rstrip('+').rstrip('*'))
df_rb

,Rk,Player,Tm,Age,Pos,G,GS,Att,Yds,TD,1D,Lng,Y/A,Y/G,Fmb
0,1,Jonathan Taylor,IND,22,RB,17,17,332,1811,18,107,83,5.5,106.5,4
1,2,Najee Harris,PIT,23,RB,17,17,307,1200,7,62,37,3.9,70.6,0
2,3,Joe Mixon,CIN,25,RB,16,16,292,1205,13,60,32,4.1,75.3,2
3,4,Antonio Gibson,WAS,23,RB,16,14,258,1037,7,65,27,4.0,64.8,6
4,5,Dalvin Cook,MIN,26,RB,13,13,249,1159,6,57,66,4.7,89.2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,367,Quez Watkins,PHI,23,WR,17,12,1,3,0,0,3,3.0,0.2,0
379,368,Derek Watt,PIT,29,fb,17,4,1,1,0,1,1,1.0,0.1,0
380,369,Preston Williams,MIA,24,wr,8,3,1,7,0,0,7,7.0,0.9,0
381,370,Andrew Wingard,JAX,25,S,15,15,1,4,0,1,4,4.0,0.3,0


In [7]:
def remove_punctuation(column, punc):
    return column.replace(punc, '', regex=True, inplace=True)

remove_punctuation(df_wr['Ctch%'], '%')

int_cols = ['Tgt', 'Rec', 'TD', 'Yds', 'Lng', 'Fmb', 'G', 'GS', '1D']
float_cols = ['Ctch%', 'Y/R', 'R/G', 'Y/Tgt', 'Y/G']

for int_col in int_cols:
    df_wr[f'{int_col}'] = df_wr[f'{int_col}'].astype(int)

for float_col in float_cols:
    df_wr[f'{float_col}'] = df_wr[f'{float_col}'].astype(float)

df_wr['Player'] = df_wr['Player'].map(lambda x: x.lstrip('*').rstrip('+').rstrip('*'))
df_wr

,Rk,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,1D,Lng,Y/Tgt,R/G,Y/G,Fmb
0,1,Cooper Kupp,LAR,28,WR,17,17,191,145,75.9,1947,13.4,16,89,59,10.2,8.5,114.5,0
1,2,Davante Adams,GNB,29,WR,16,16,169,123,72.8,1553,12.6,11,84,59,9.2,7.7,97.1,0
2,3,Tyreek Hill,KAN,27,WR,17,16,159,111,69.8,1239,11.2,9,75,75,7.8,6.5,72.9,2
3,4,Justin Jefferson,MIN,22,WR,17,17,167,108,64.7,1616,15.0,10,75,56,9.7,6.4,95.1,1
4,5,Mark Andrews,BAL,26,te,17,9,153,107,69.9,1361,12.7,9,75,43,8.9,6.3,80.1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522,507,Christian Wilkins,MIA,26,DT,17,17,1,1,100.0,1,1.0,1,1,1,1.0,0.1,0.1,1
523,508,Seth Williams,DEN,21,wr,2,1,1,1,100.0,34,34.0,0,1,34,34.0,0.5,17.0,0
524,509,Trayveon Williams,CIN,24,NaN,5,0,2,1,50.0,4,4.0,0,0,4,2.0,0.2,0.8,0
526,510,Easop Winston,NOR,25,NaN,3,1,1,1,100.0,5,5.0,0,0,5,5.0,0.3,1.7,0


In [8]:
df_qb.fillna(0, inplace=True)

int_cols = ['Cmp', 'Att', 'Yds', 'TD', 'Int', '1D', 'Lng', 'GS', 'Sk', 'Yds.1', '4QC', 'GWD']
float_cols = ['Cmp%', 'TD%', 'Int%', 'Y/A', 'AY/A', 'Y/C', 'Y/G', 'Rate', 'QBR', 'Rate', 'NY/A', 'ANY/A', 'Sk%']

for int_col in int_cols:
    df_qb[f'{int_col}'] = df_qb[f'{int_col}'].astype(int)

for float_col in float_cols:
    df_qb[f'{float_col}'] = df_qb[f'{float_col}'].astype(float)

df_qb['Player'] = df_qb['Player'].map(lambda x: x.lstrip('*').rstrip('*'))
df_qb

,Rk,Player,Tm,Age,Pos,G,GS,QBrec,Cmp,Att,Cmp%,Yds,TD,TD%,Int,Int%,1D,Lng,Y/A,AY/A,Y/C,Y/G,Rate,QBR,Sk,Yds.1,Sk%,NY/A,ANY/A,4QC,GWD
0,1,Tom Brady,TAM,44,QB,17,17,13-4-0,485,719,67.5,5316,43,6.0,12,1.7,269,62,7.4,7.8,11.0,312.7,102.1,68.1,22,144,3.0,6.98,7.41,3,5
1,2,Justin Herbert,LAC,23,QB,17,17,9-8-0,443,672,65.9,5014,38,5.7,15,2.2,256,72,7.5,7.6,11.3,294.9,97.7,65.6,31,214,4.4,6.83,6.95,5,5
2,3,Matthew Stafford,LAR,33,QB,17,17,12-5-0,404,601,67.2,4886,41,6.8,17,2.8,233,79,8.1,8.2,12.1,287.4,102.9,63.8,30,243,4.8,7.36,7.45,3,4
3,4,Patrick Mahomes,KAN,26,QB,17,17,12-5-0,436,658,66.3,4839,37,5.6,13,2.0,260,75,7.4,7.6,11.1,284.6,98.5,62.2,28,146,4.1,6.84,7.07,3,3
4,5,Derek Carr,LVR,30,QB,17,17,10-7-0,428,626,68.4,4804,23,3.7,14,2.2,217,61,7.7,7.4,11.2,282.6,94.0,52.4,40,241,6.0,6.85,6.60,3,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,111,Cordarrelle Patterson,ATL,30,RB,16,13,0,0,1,0.0,0,0,0.0,0,0.0,0,0,0.0,0.0,0.0,0.0,39.6,2.9,0,0,0.0,0.00,0.00,0,0
114,112,Brett Rypien,DEN,25,0,1,0,0,0,2,0.0,0,0,0.0,0,0.0,0,0,0.0,0.0,0.0,0.0,39.6,1.1,0,0,0.0,0.00,0.00,0,0
115,113,D'Andre Swift,DET,22,rb,13,4,0,0,1,0.0,0,0,0.0,0,0.0,0,0,0.0,0.0,0.0,0.0,39.6,0.5,0,0,0.0,0.00,0.00,0,0
116,114,Albert Wilson,MIA,29,wr,14,5,0,0,1,0.0,0,0,0.0,0,0.0,0,0,0.0,0.0,0.0,0.0,39.6,0.0,1,1,50.0,-0.50,-0.50,0,0


#### create TE df by grabbing te data from wr df

In [9]:
df_te = df_wr.loc[(df_wr['Pos'] == 'TE')].copy()
df_te

,Rk,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,1D,Lng,Y/Tgt,R/G,Y/G,Fmb
12,13,Travis Kelce,KAN,32,TE,16,16,134,92,68.7,1125,12.2,9,63,69,8.4,5.8,70.3,1
22,23,Dalton Schultz,DAL,25,TE,17,15,104,78,75.0,808,10.4,8,43,32,7.8,4.6,47.5,0
36,36,George Kittle,SFO,28,TE,14,14,94,71,75.5,910,12.8,6,42,48,9.7,5.1,65.0,2
40,40,Noah Fant,DEN,24,TE,16,16,90,68,75.6,670,9.9,4,29,35,7.4,4.3,41.9,0
41,41,Kyle Pitts,ATL,21,TE,17,15,110,68,61.8,1026,15.1,1,43,61,9.3,4.0,60.4,0
49,49,Tyler Conklin,MIN,26,TE,17,15,87,61,70.1,593,9.7,3,26,40,6.8,3.6,34.9,1
50,50,Tyler Higbee,LAR,28,TE,15,15,85,61,71.8,560,9.2,5,33,37,6.6,4.1,37.3,1
51,51,T.J. Hockenson,DET,24,TE,12,12,84,61,72.6,583,9.6,4,32,33,6.9,5.1,48.6,0
53,53,Cole Kmet,CHI,22,TE,17,17,93,60,64.5,612,10.2,0,29,25,6.6,3.5,36.0,0
58,58,Dallas Goedert,PHI,26,TE,15,14,76,56,73.7,830,14.8,4,42,45,10.9,3.7,55.3,1


### Create a dataframe with all player names and their corresponding team names (full team name, abbreviation, mascot)

In [10]:
df_wr.shape, df_qb.shape, df_rb.shape, df_te.shape

((511, 19), (115, 31), (371, 15), (29, 19))

In [11]:
dfs = [df_wr, df_qb, df_rb, df_te]
player_info = pd.concat([player_info.squeeze() for player_info in dfs], ignore_index=True)

In [12]:
player_info = player_info[['Player', 'Tm', 'Pos']].copy()

In [13]:
player_info.rename(columns = {'Tm':'Team_Name_Abbrev'}, inplace = True)

In [14]:
player_info.head()

,Player,Team_Name_Abbrev,Pos
0,Cooper Kupp,LAR,WR
1,Davante Adams,GNB,WR
2,Tyreek Hill,KAN,WR
3,Justin Jefferson,MIN,WR
4,Mark Andrews,BAL,te


In [15]:
player_info.shape

(1026, 3)

In [16]:
player_info = player_info.drop_duplicates()
player_info.head()

,Player,Team_Name_Abbrev,Pos
0,Cooper Kupp,LAR,WR
1,Davante Adams,GNB,WR
2,Tyreek Hill,KAN,WR
3,Justin Jefferson,MIN,WR
4,Mark Andrews,BAL,te


In [17]:
player_info_dict = pd.Series(player_info.Team_Name_Abbrev.values,index=player_info.Player).to_dict()
# create dictionary with {player : team abbreviation}

In [18]:
player_info_dict['Cooper Kupp']

'LAR'

In [19]:
Team_Abbreviations_Dict = {
    'Arizona Cardinals': 'ARI',
    'Atlanta Falcons': 'ATL',
    'Baltimore Ravens': 'BAL',
    'Buffalo Bills' : 'BUF',
    'Carolina Panthers': 'CAR',
    'Chicago Bears': 'CHI',
    'Cincinnati Bengals': 'CIN',
    'Cleveland Browns': 'CLE',
    'Dallas Cowboys': 'DAL',
    'Denver Broncos': 'DEN',
    'Detroit Lions': 'DET',
    'Green Bay Packers': 'GNB',
    'Houston Texans': 'HOU',
    'Indianapolis Colts': 'IND',
    'Jacksonville Jaguars': 'JAX',
    'Kansas City Chiefs': 'KAN',
    'Miami Dolphins': 'MIA',
    'Minnesota Vikings': 'MIN',
    'New England Patriots': 'NWE',
    'New Orleans Saints': 'NO',
    'NY Giants': 'NYG',
    'NY Jets': 'NYJ',
    'Las Vegas Raiders': 'LVR',
    'Philadelphia Eagles': 'PHI',
    'Pittsburgh Steelers': 'PIT',
    'Los Angeles Chargers': 'LAC',
    'San Francisco 49ers': 'SFO',
    'Seattle Seahawks': 'SEA',
    'Los Angeles Rams': 'LAR',
    'Tampa Bay Buccaneers': 'TAM',
    'Tennessee Titans': 'TEN',
    'Washington Commanders': 'WAS'
}

In [20]:
def get_key(val):
    for key, value in Team_Abbreviations_Dict.items():
         if val == value:
            return key
 
    return f'key does not exist --> {val}'

In [21]:
player_info['Team_Name_Full'] = player_info['Team_Name_Abbrev'].map(get_key)

In [22]:
player_info['Team_Name_Mascot'] = player_info['Team_Name_Full'].map(lambda x: x.split()[-1])

In [23]:
player_info.head(15)

,Player,Team_Name_Abbrev,Pos,Team_Name_Full,Team_Name_Mascot
0,Cooper Kupp,LAR,WR,Los Angeles Rams,Rams
1,Davante Adams,GNB,WR,Green Bay Packers,Packers
2,Tyreek Hill,KAN,WR,Kansas City Chiefs,Chiefs
3,Justin Jefferson,MIN,WR,Minnesota Vikings,Vikings
4,Mark Andrews,BAL,te,Baltimore Ravens,Ravens
5,Diontae Johnson,PIT,WR,Pittsburgh Steelers,Steelers
6,Keenan Allen,LAC,WR,Los Angeles Chargers,Chargers
7,Jaylen Waddle,MIA,WR,Miami Dolphins,Dolphins
8,Stefon Diggs,BUF,WR,Buffalo Bills,Bills
9,Hunter Renfrow,LVR,wr,Las Vegas Raiders,Raiders


In [25]:
player_info.to_csv(f'./clean_data/player_info.csv', index=0)